# Invariant Measure of the 2d Rotated Well

In [9]:
import xarray as xr
import matplotlib.pyplot as plt
import numpy as np
import os
import seaborn as sns
import pandas as pd
sns.set_theme(style="darkgrid")
import matplotlib as mpl


## Loading Data

In [ ]:
data_dir = 'Data/eps0_1/'
alpha_dirs = [data_dir + a +'/' for a  in os.listdir(data_dir)]
# Order dirs by alpha
alpha_order = [3, 2, 1, 0]
alpha_dirs = [alpha_dirs[i] for i in alpha_order]

In [ ]:
alpha_dirs

In [ ]:
cold_data = []
hot_data = []
for a in alpha_dirs:
    ds = xr.open_dataset(a + '/hot-ensemble/1.nc')
    hot_data.append(ds)
    ds = xr.open_dataset(a + '/cold-ensemble/1.nc')
    cold_data.append(ds)

In [ ]:
cold_data

In [ ]:
dfs = []
for ds in hot_data:
    df = pd.DataFrame({'alpha': ds.alpha, 'x': ds.x.values.flatten(), 'y': ds.y.values.flatten()})
    dfs.append(df)
d = pd.concat(dfs)
print('Plotting')
%time sns.jointplot(data=d, x='x', y='y', hue='alpha', kind="hist", bins=50)

In [ ]:
# Plotting Invariant Measure: X and Y Marginals

fig, axes = plt.subplots(ncols=2, figsize=(15, 8))


for i in range(len(hot_data)):
   
    # Collecting both hot and cold data

    h_ds = hot_data[i]
    c_ds = cold_data[i]
    alpha = c_ds.alpha
    
    x_data = np.append(c_ds.x.values.flatten(), (h_ds.x.values.flatten()))
    y_data = np.append(c_ds.y.values.flatten(), (h_ds.y.values.flatten()))

    # Plotting the histogram
    axes[0].hist(x_data, histtype='step', bins=500, label=f'{alpha:.2f}', density='True')
    axes[1].hist(y_data, histtype='step', bins=500, label=f'{alpha:.2f}', density='True')


for ax in axes:
    ax.legend(title= '$\\alpha$')
    ax.set_ylabel('$\log(\\rho$)')
    ax.set_yscale('log')

axes[0].set_title('Invariant Measure: X Density')
axes[0].set_xlabel('X')
axes[1].set_xlabel('Y')
axes[1].set_title('Invariant Measure: Y Density')
fig.savefig('Invariant-Meaure-X-and-Y-Marginals.pdf')

print(f'Plots made using {len(x_data):.2e} data points')
fig.show()

In [ ]:
# Function for Streamfield background

def streamfield_background(alpha, ax): 
    # Plot Misc
    ax.set_xlim((-2, 2))
    ax.set_ylim((-1, 1))
    ax.set_xlabel('x')
    ax.set_ylabel('y')


    # alpha label
    ax.text(-1.2, 0.8, fr'$\alpha = {alpha:.2f}$', fontsize=15, bbox={'facecolor': '1', 'pad': 10}) #

    # Gradient Arrows
    x = np.linspace(-2, 2, 100)
    y = np.linspace(-1, 1, 100)
    X, Y = np.meshgrid(x, y)
    vx = X*(X**2 - 1) - 2 * alpha * Y 
    vy = alpha *X*(X**2 - 1) + 2 * Y 
    speed = np.sqrt(vx**2 + vy**2)
    ax.streamplot(x, y, -vx, -vy, color='0')
    # ax.pcolormesh(X, Y, speed, cmap = 'Blues') 

    # Fixed Points
    cold_point = plt.Circle((-1, 0), 0.1, color='b', alpha=0.5)
    mid_point = plt.Circle((0, 0), 0.1, color='g', alpha=0.5)
    hot_point = plt.Circle((1, 0), 0.1, color='r', alpha=0.5)
#     ax.add_patch(cold_point)
# #     ax.add_patch(mid_point)
#     ax.add_patch(hot_point)
    return ax



fig, axes = plt.subplots(2, 2, figsize = (16, 16))


# Plotting Histogram of Integration results

for i in range(len(hot_data)):
   
    # Collecting both hot and cold data

    h_ds = hot_data[i]
    c_ds = cold_data[i]
    alpha = c_ds.alpha
    
    x_data = np.append(c_ds.x.values.flatten(), (h_ds.x.values.flatten()))
    y_data = np.append(c_ds.y.values.flatten(), (h_ds.y.values.flatten()))
    
    # Plotting Histogram
    ax = axes.flatten()[i]
    h = ax.hist2d(x_data, y_data, range= [[-2, 2], [-1, 1]], bins=100, cmap=mpl.cm.Blues, norm=mpl.colors.LogNorm(), density='True')
    cbar = fig.colorbar(h[3], ax=ax) #h[3] is count
    cbar.ax.set_ylabel('$\log (\\rho)$')
    
    streamfield_background(alpha, ax)


fig.suptitle('Effect of $\\alpha$ on Invariant Measure', fontsize=16)
fig.savefig('Invariant-Measure-Alpha-Effect.pdf')
fig.show()


In [ ]:
# Stochastic Integration Results
dirs = [f'../Stochastic-Model/Data/eps0_05/alpha_{a}/cold-ensemble/Transition-Data/' for a in ['0_0', '1_0']]

integration_results = []

for d in dirs:
    transitions = []
    print(f'Opening {d}')
    for f in tqdm(os.listdir(d)):
        if (f[-3:] == '.nc'):
            ds = xr.open_dataset(d + f)
            transitions.append(ds)
    integration_results.append(transitions)
    
alpha0_results, alpha1_results = integration_results

Opening ../Stochastic-Model/Data/eps0_05/alpha_0_0/cold-ensemble/Transition-Data/


In [ ]:
alpha1_results